# ETL Notebook

Loads the raw json data and writes it out as parquet.  NOTE this will overwrite any existing parquet data.

In [1]:
# import the etl module
from tailor import etl

In [2]:
# load the events, show how many records we loaded
events_df = etl.load_events(spark, 'raw_data/rocdev')
events_df.count()

168340

In [3]:
# clear out any existing parquet event data
!rm -rf parquet_data/rocdev/events.parquet
!mkdir -p parquet_data/rocdev

We write to [parquet](https://parquet.apache.org/) format, which is a binary format that is fairly common for spark.

Parquet + Spark supports [partitioning](https://en.wikipedia.org/wiki/Partition_(database)) and if this were an ETL process for a larger-scale system, we would probably want to partition by date (and maybe channel) so that we could incrementally load/reload data.  To partition this data, we would simply need to change the line below to `events_df.write.partitionBy("date").parquet(...)`.  For this small data set, which is fairly sparse when looked at date-over-date, partitioning by date actually leads to very inefficient storage (500MB vs 25MB!).

In [4]:
# write the event parquet
events_df.write.parquet('parquet_data/rocdev/events.parquet')

In [5]:
# let's see what the output looks like on disk
!ls -lah parquet_data/rocdev/events.parquet | head -n 20

total 50576
drwxr-xr-x  20 dswain  staff   640B Apr  1 10:26 .
drwxr-xr-x   5 dswain  staff   160B Apr  1 10:26 ..
-rw-r--r--   1 dswain  staff     8B Apr  1 10:26 ._SUCCESS.crc
-rw-r--r--   1 dswain  staff    20K Apr  1 10:26 .part-00000-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    23K Apr  1 10:26 .part-00001-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    21K Apr  1 10:26 .part-00002-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    22K Apr  1 10:26 .part-00003-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    21K Apr  1 10:26 .part-00004-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    23K Apr  1 10:26 .part-00005-37207290-68ad-46df-b104-7cc85b94d1b9-c000.snappy.parquet.crc
-rw-r--r--   1 dswain  staff    24K Apr  1 10:26 .part-00006-37207290-68ad-46df-b104-7cc85

In [6]:
# how much disk space does this use?
!du -h -d 0 raw_data/rocdev
!du -h -d 1 parquet_data/rocdev

 89M	raw_data/rocdev
484K	parquet_data/rocdev/users.parquet
 25M	parquet_data/rocdev/events.parquet
136K	parquet_data/rocdev/channels.parquet
 25M	parquet_data/rocdev


In [7]:
# load the user data
users_df = etl.load_users(spark, 'raw_data/rocdev')

In [8]:
# make space for user parquet
!rm -rf parquet_data/rocdev/users.parquet
!mkdir -p parquet_data/rocdev/

In [9]:
# write user parquet
users_df.write.parquet('parquet_data/rocdev/users.parquet')

In [10]:
# lather, rinse, repeat with channels
channels_df = etl.load_channels(spark, 'raw_data/rocdev')

In [11]:
!rm -rf parquet_data/rocdev/channels.parquet
!mkdir -p parquet_data/rocdev/

In [12]:
channels_df.write.parquet('parquet_data/rocdev/channels.parquet')

In [13]:
!du -h -d 1 parquet_data/rocdev

484K	parquet_data/rocdev/users.parquet
 25M	parquet_data/rocdev/events.parquet
136K	parquet_data/rocdev/channels.parquet
 25M	parquet_data/rocdev


In [14]:
# let's read back the events just as a sanity check
df = spark.read.parquet('parquet_data/rocdev/events.parquet')
df.count()

168340

In [15]:
# what does the schema look like?
df.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- actions: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- confirm: struct (nullable = true)
 |    |    |    |    |    |-- dismiss_text: string (nullable = true)
 |    |    |    |    |    |-- ok_text: string (nullable = true)
 |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |    |-- title: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- style: string (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- audio_html: string (nullable = true)
 |    |    |-- audio_html_height: long (nullable = true)
 |    |    |-- audio_html_width: long (

In [16]:
# what does the users schema look like?
users_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- deleted: boolean (nullable = true)
 |-- id: string (nullable = true)
 |-- is_admin: boolean (nullable = true)
 |-- is_app_user: boolean (nullable = true)
 |-- is_bot: boolean (nullable = true)
 |-- is_owner: boolean (nullable = true)
 |-- is_primary_owner: boolean (nullable = true)
 |-- is_restricted: boolean (nullable = true)
 |-- is_ultra_restricted: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- profile: struct (nullable = true)
 |    |-- always_active: boolean (nullable = true)
 |    |-- api_app_id: string (nullable = true)
 |    |-- avatar_hash: string (nullable = true)
 |    |-- bot_id: string (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- display_name_normalized: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- fields: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- image_1024: string (nullable = true)
 |    |-- image_192:

In [17]:
# channels?
channels_df.printSchema()

root
 |-- created: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- id: string (nullable = true)
 |-- is_archived: boolean (nullable = true)
 |-- is_general: boolean (nullable = true)
 |-- members: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- pins: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: long (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- owner: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- user: string (nullable = true)
 |-- purpose: struct (nullable = true)
 |    |-- creator: string (nullable = true)
 |    |-- last_set: string (nullable = true)
 |    |-- value: string (nullable = true)
 |-- topic: struct (nullable = true)
 |    |-- creator: string (nullable = true)
 |    |-- last_set: string (nullable = true)
 |    |-- value: string (nullable = true)

